In [1]:
import pandas as pd, tensorflow as tf, numpy as np, spacy
from tensorflow import keras
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from scikeras.wrappers import KerasRegressor, KerasClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from keras import metrics
from sklearn.model_selection import train_test_split

2024-11-16 20:55:36.137150: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-16 20:55:36.142475: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-16 20:55:36.194696: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-16 20:55:36.256963: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731801336.303974   27021 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731801336.31

In [2]:
headlines_df = pd.read_csv('new_headlines.csv')

headlines = headlines_df.copy()

headlines_df.head()

,date,fortnight,fortnight_day,section,title,diff
0,2010-01-03,0,5,business,China and the other Brics will rebuild a new w...,19.26
1,2010-01-13,1,1,business,US taxpayers gave banks kiss of life Too dange...,-48.18
2,2010-01-27,2,1,business,Tullow takes a tumble as FTSE falters on globa...,-29.37
3,2010-02-10,3,1,business,FTSE moves higher on Greek hopes and Wall Stre...,37.11
4,2010-02-24,4,1,business,FTSE falters ahead of Bernanke testimony HSBC ...,40.37


In [3]:
headlines = headlines_df.copy()

headlines_df.head()

,date,fortnight,fortnight_day,section,title,diff
0,2010-01-03,0,5,business,China and the other Brics will rebuild a new w...,19.26
1,2010-01-13,1,1,business,US taxpayers gave banks kiss of life Too dange...,-48.18
2,2010-01-27,2,1,business,Tullow takes a tumble as FTSE falters on globa...,-29.37
3,2010-02-10,3,1,business,FTSE moves higher on Greek hopes and Wall Stre...,37.11
4,2010-02-24,4,1,business,FTSE falters ahead of Bernanke testimony HSBC ...,40.37


In [4]:
nlp = spacy.load('en_core_web_md')

In [5]:
vectors = headlines_df['title'].apply(lambda x: nlp(x).vector)

vectors

0      [-1.3010186, 0.26826838, -2.0974205, 0.7587583...
1      [-1.1559626, -0.06732634, -2.267186, 0.8647494...
2      [-1.1932443, 0.13201025, -2.1287062, 0.9592638...
3      [-1.2596585, 0.075955436, -1.6795421, 0.978719...
4      [-1.4499966, -0.07736405, -2.2833703, 0.963610...
                             ...                        
381    [-1.155147, 0.10391867, -1.9603742, 0.46184877...
382    [-1.0387702, 0.1456095, -1.8198619, 0.57597536...
383    [-1.3400451, 0.16482167, -2.0387025, 0.3914396...
384    [-1.0087184, -0.122355685, -2.0073137, 0.57646...
385    [-1.0858915, -0.10410232, -2.009698, 0.4340306...
Name: title, Length: 386, dtype: object

In [6]:
vectors_df = pd.DataFrame(vectors.tolist())

headlines = pd.concat([headlines_df.drop(columns=['date', 'title']), vectors_df], axis=1)

headlines['up'] = headlines['diff'].apply(lambda x: 1 if x >= 0 else 0)

headlines = headlines[[column for column in headlines.columns if column != 'diff'] + ['diff']]

headlines.tail()

,fortnight,fortnight_day,section,0,1,2,3,4,5,6,...,292,293,294,295,296,297,298,299,up,diff
381,381,2,business,-1.155147,0.103919,-1.960374,0.461849,3.318084,-0.002882,1.132856,...,-0.585809,-0.061483,-0.736804,1.954284,0.673725,-1.650408,-1.641880,1.828687,1,421.35
382,382,1,business,-1.038770,0.145609,-1.819862,0.575975,3.469416,0.146417,1.204553,...,-0.696766,-0.087798,-1.068300,1.359791,0.661663,-1.281119,-1.825274,1.458141,0,-100.78
383,383,1,business,-1.340045,0.164822,-2.038702,0.391440,3.364496,0.149060,1.374547,...,-0.926733,-0.053264,-0.700962,1.507293,0.631234,-1.209178,-1.860705,2.047713,1,98.19
384,384,1,business,-1.008718,-0.122356,-2.007314,0.576462,3.385915,0.004756,1.106388,...,-0.637391,-0.204573,-0.782671,1.755281,0.670324,-1.571848,-1.692215,1.701142,1,91.28
385,385,1,business,-1.085891,-0.104102,-2.009698,0.434031,3.415793,0.179098,1.359401,...,-0.891932,-0.264010,-0.838934,1.590277,0.565512,-1.467564,-1.595881,2.276219,1,132.93


In [7]:
train, test = train_test_split(headlines, test_size=0.33, shuffle=True, stratify=headlines.up)

X_train, y_train_class, y_train_reg = train.iloc[:, :-2], train['up'], train['diff']
X_test, y_test_class, y_test_reg = test.iloc[:, :-2], test['up'], test['diff']

X_train.columns, X_test.columns = X_train.columns.astype(str), X_test.columns.astype(str)

X_train.head()

,fortnight,fortnight_day,section,0,1,2,3,4,5,6,...,290,291,292,293,294,295,296,297,298,299
115,115,1,technology,-0.661777,-0.078106,-1.733615,0.880859,3.162097,-0.656119,0.989584,...,-0.035826,-0.090304,0.442765,0.227002,-1.126105,1.670934,1.103448,-1.086807,-1.817292,1.397708
172,172,3,business,-1.203904,0.615625,-2.766694,2.407309,3.372231,-0.238669,0.457245,...,0.670748,-0.135393,-0.406645,-0.073468,-1.526001,2.148188,1.258813,-1.953541,-1.515833,1.037346
116,116,3,business,-0.877595,0.810675,-2.585556,1.069144,3.134650,0.221715,0.889277,...,0.755815,-0.296750,0.144194,-0.098609,-0.931285,1.412226,0.987263,-0.899398,-2.444102,2.105026
139,139,1,business,-1.307627,0.217559,-1.747570,1.025495,3.469143,-0.018591,0.907349,...,0.675769,-0.697050,-0.454898,0.465367,-1.123608,1.607500,1.358431,-1.392256,-1.832958,1.780417
145,145,1,business,-1.123162,0.221153,-1.785539,0.820365,3.338564,0.152796,0.971581,...,0.545596,-0.767723,-0.241038,0.199298,-0.994092,1.222365,0.947613,-1.047959,-1.734836,1.608883


In [8]:
colunas_numericas = X_train.select_dtypes(include='number').columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), colunas_numericas),
        ('cat', OneHotEncoder(), ['section'])
    ]
)

pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

X_train = pipeline.fit_transform(X_train)
X_test = pipeline.transform(X_test)

X_train

array([[-0.74618237, -0.22267542,  1.83732958, ..., -1.33414286,
         0.        ,  1.        ],
       [-0.23730842,  1.75836793, -0.3318409 , ..., -2.56911922,
         1.        ,  0.        ],
       [-0.73725475,  1.75836793,  0.97379621, ...,  1.08986436,
         1.        ,  0.        ],
       ...,
       [ 0.23585508, -0.22267542,  0.63219538, ..., -0.69955645,
         1.        ,  0.        ],
       [ 1.0571955 , -0.22267542,  0.43300495, ..., -1.69556297,
         1.        ,  0.        ],
       [ 0.21799986, -0.22267542,  0.28001508, ...,  0.97329536,
         1.        ,  0.        ]])

In [9]:
def create_model():

    model = keras.models.Sequential([
        keras.layers.Input(shape=[X_train.shape[1]]),
        keras.layers.Dense(100, activation='relu', kernel_regularizer=l2(0.1)),
        keras.layers.Dense(100, activation='relu', kernel_regularizer=l2(0.1)),
        keras.layers.Dense(100, activation='relu', kernel_regularizer=l2(0.1)),
        keras.layers.Dense(100, activation=keras.activations.swish),
        keras.layers.Dense(1, activation='sigmoid')  # Saída com 2 classes e ativação softmax para classificação
    ])

    model.compile(loss='mean_squared_error', metrics=['accuracy'])
    return model

early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

keras_model = KerasClassifier(model=create_model, callbacks=[early_stopping])

param_grid = {
    'optimizer': ['sgd', 'rmsprop', 'adam'],
    'epochs': [50, 100],
}

cv = StratifiedKFold(n_splits=10)

# Configurando o GridSearchCV
grid = GridSearchCV(estimator=keras_model, param_grid=param_grid, cv=cv, scoring='balanced_accuracy', n_jobs=3, verbose=1)

grid_result = grid.fit(X_train, y_train_class)

Fitting 10 folds for each of 6 candidates, totalling 60 fits


2024-11-16 20:56:10.422148: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-16 20:56:10.423088: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-16 20:56:10.426468: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-16 20:56:10.427232: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-16 20:56:10.428126: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your

Epoch 1/50
Epoch 1/50
Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5789 - loss: 32.0834
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5494 - loss: 32.4207
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5470 - loss: 32.4688
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5896 - loss: 24.8894 
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6490 - loss: 24.9634 
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6477 - loss: 24.9013 
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6148 - loss: 20.5209 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6224 - loss: 20.4441
Epoch 4/50
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6249 - loss: 20.4516
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6513 - loss: 16.9863 
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6200 - loss: 16.9258 
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy:

/home/vitor/anaconda3/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
/home/vitor/anaconda3/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
/home/vitor/anaconda3/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6135 - loss: 11.5081 
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6519 - loss: 11.4469 
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6593 - loss: 11.4379 
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6131 - loss: 9.3500 
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6374 - loss: 9.2994 
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6405 - loss: 9.2875 
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6745 - loss: 7.50245
Epoch 9/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6366 - loss: 7.4733 
Epoch 9/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6384 - loss: 7.4567 
Epoch 9/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6354 - loss: 5.96832
Epoch 10/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6268 - loss: 5.9317
1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.6250 - loss: 6.2453Epoch 10/50
8/8 ━━━━━━━━

...

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6169 - loss: 1.1589 
Epoch 16/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6004 - loss: 11.3923 
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6344 - loss: 1.5563 
Epoch 15/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6154 - loss: 0.8681
Epoch 17/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6035 - loss: 9.2510 
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6224 - loss: 1.1663
Epoch 16/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6319 - loss: 0.6508
Epoch 18/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6568 - loss: 7.4182 
Epoch 9/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6330 - loss: 0.8697
Epoch 17/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6439 - loss: 0.4947
Epoch 19/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6154 - loss: 5.89644
Epoch 10/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5806 - loss: 0.6697
E

2024-11-16 20:57:41.363143: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6045 - loss: 31.85666
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6325 - loss: 23.9258
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6736 - loss: 19.2347
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6409 - loss: 15.5464 
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6557 - loss: 12.4826
Epoch 6/50
3/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6927 - loss: 10.33041

/home/vitor/anaconda3/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6585 - loss: 9.9196 
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6283 - loss: 7.7859
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6537 - loss: 6.0149
Epoch 9/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6669 - loss: 4.5756
Epoch 10/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6214 - loss: 3.4365
Epoch 11/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6211 - loss: 2.5377
Epoch 12/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6088 - loss: 1.8535
Epoch 13/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6753 - loss: 1.3265
Epoch 14/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6578 - loss: 0.9571
Epoch 15/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6363 - loss: 0.6968
Epoch 16/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6499 - loss: 0.5107
Epoch 17/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6242 - loss: 0.3939
Epoch 18/50
9/9 ━━

In [10]:
best = grid_result.best_estimator_

best.fit(X_train, y_train_class)

Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5811 - loss: 31.9226 
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6514 - loss: 23.9804
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6066 - loss: 19.2997
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6156 - loss: 15.6017
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6670 - loss: 12.5243
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6189 - loss: 9.9672 
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6264 - loss: 7.8198
Epoch 8/50


/home/vitor/anaconda3/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6231 - loss: 6.0517
Epoch 9/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6759 - loss: 4.6009 
Epoch 10/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6331 - loss: 3.4598
Epoch 11/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6762 - loss: 2.5443
Epoch 12/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6077 - loss: 1.8693
Epoch 13/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6315 - loss: 1.3460
Epoch 14/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6389 - loss: 0.9673
Epoch 15/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6069 - loss: 0.7081 
Epoch 16/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6443 - loss: 0.5141
Epoch 17/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6564 - loss: 0.3871
Epoch 18/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6374 - loss: 0.3126
Epoch 19/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6238 - loss: 0.2702
Epoch 20/50
9/9

KerasClassifier(
	model=<function create_model at 0x7efca12ecea0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=sgd
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=[<keras.src.callbacks.early_stopping.EarlyStopping object at 0x7efc9acd20c0>]
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=50
	class_weight=None
)

In [11]:
print(classification_report(y_test_class, best.predict(X_test)))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        47
           1       0.63      1.00      0.78        81

    accuracy                           0.63       128
   macro avg       0.32      0.50      0.39       128
weighted avg       0.40      0.63      0.49       128



/home/vitor/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/vitor/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/vitor/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:
model = keras.models.Sequential([
    keras.layers.Input(shape=[X_train.shape[1]]),
    keras.layers.Dense(250, activation='relu'),
    keras.layers.Dense(250, activation='relu'),
    keras.layers.Dense(250, activation='relu'),
    keras.layers.Dense(1, activation='linear')
])

model.compile(optimizer='adam', 
              loss='mean_squared_error', 
              metrics=[
                metrics.MeanAbsoluteError(),
                metrics.R2Score()
              ])

# Criando o KerasRegressor
keras_model = KerasRegressor(model=model)

In [13]:
param_grid = {
    'optimizer': ['rmsprop', 'sgd'],
    'epochs': [10, 50, 100],
}

# Criando TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=3)

# Configurando o GridSearchCV
grid = GridSearchCV(estimator=keras_model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=1)

NameError: name 'TimeSeriesSplit' is not defined

In [ ]:
grid_result = grid.fit(X_train, y_train_reg)

# Mostrando os resultados
print("Melhores parâmetros:", grid_result.best_params_)
print("Melhor pontuação (neg_mean_squared_error):", grid_result.best_score_)

/home/vitor/anaconda3/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 18 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 7988.2720 - mean_absolute_error: 56.5072 - r2_score: -0.3032  
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8414.2959 - mean_absolute_error: 55.0128 - r2_score: -0.2497 
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7230.9360 - mean_absolute_error: 51.5997 - r2_score: -0.1644 
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7808.1670 - mean_absolute_error: 52.7018 - r2_score: -0.0696  
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6445.5063 - mean_absolute_error: 48.0566 - r2_score: 0.0773 
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4760.3926 - mean_absolute_error: 43.8660 - r2_score: 0.2040
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4964.6440 - mean_absolute_error: 44.0496 - r2_score: 0.2903 
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3778.5884 - mean_absolute_error: 39.3233 - r2_score: 0.3699 
Epoch 9/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 395

In [ ]:
best = grid_result.best_estimator_

best.fit(X_train, y_train_reg)

Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5628 - mean_absolute_error: 0.5363 - r2_score: 0.9999 
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3615 - mean_absolute_error: 0.3858 - r2_score: 1.0000
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2706 - mean_absolute_error: 0.3299 - r2_score: 1.0000 
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3279 - mean_absolute_error: 0.3777 - r2_score: 1.0000
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2934 - mean_absolute_error: 0.3810 - r2_score: 1.0000
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1428 - mean_absolute_error: 0.2605 - r2_score: 1.0000
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2369 - mean_absolute_error: 0.3592 - r2_score: 1.0000 
Epoch 8/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5338 - mean_absolute_error: 0.6223 - r2_score: 0.9998 
Epoch 9/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2559 - mean_absolute_error: 0.694

KerasRegressor(
	model=<Sequential name=sequential_2, built=True>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=100
)

In [ ]:
y_pred = best.predict(X_train)

list(zip(y_pred, y_test_reg))

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 976us/step


[(31.533302, 29.47),
 (115.4779, -15.35),
 (-31.307972, 4.3),
 (30.291887, 274.42),
 (11.800745, 9.92),
 (26.96454, -138.45),
 (1.7811265, 33.32),
 (12.8008995, 42.84),
 (-9.702535, 14.21),
 (-44.86024, -13.26),
 (65.17258, 7.88),
 (103.45085, 192.76),
 (25.90196, -175.63),
 (-42.172924, -130.75),
 (-12.614533, 33.79),
 (11.0579405, 49.33),
 (-23.487999, 23.49),
 (4.596569, 49.36),
 (4.595581, -33.67),
 (3.1388211, 12.62),
 (66.55422, 21.06),
 (8.51822, 12.18),
 (129.51888, 25.69),
 (143.83522, -163.96),
 (36.435158, 0.25),
 (-2.694212, 41.77),
 (-12.585163, 4.76),
 (39.526466, 64.95),
 (102.279045, 39.06),
 (19.27994, 45.06),
 (-13.265339, 18.54),
 (-69.33221, -227.63),
 (40.9139, 72.44),
 (62.446117, -32.19),
 (-38.602287, 45.6),
 (-2.1704469, 122.96),
 (16.026138, 64.14),
 (7.3922896, 7.37),
 (48.780487, 74.49),
 (70.72937, -125.18),
 (-78.8157, -206.78),
 (94.90867, 18.47),
 (11.912471, -41.25),
 (8.5099125, 106.07),
 (-14.36758, -6.64),
 (42.31999, 68.02),
 (22.283337, -56.67),
 (